# Setup

In [6]:
import os
import openai
import yaml
os.chdir("../../../../")
 
# openai.api_key = os.environ["OPENAI_API_KEY"]
if os.path.isfile('app/config.yaml'):
    with open('app/config.yaml', 'r') as file:
        cfg = yaml.safe_load(file)
        openai.api_key = cfg["openapi_key"]

def llm(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]



In [7]:
llm("test")

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

In [ ]:
import wikienv, wrappers
env = wikienv.WikiEnv()
env = wrappers.FactCheckWrapper(env)

template = 'Your task is to perform only one next action in the progress to check if the Observation SUPPORTS or REFUTES a Claim, or if there is NOT ENOUGH INFORMATION. Here are some examples:\nExample 1: Claim: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.\nThought 1: I need to search Nikolaj Coster-Waldau and find if he has worked with the Fox Broadcasting Company.\nAction 1: Search[Nikolaj Coster-Waldau]\nObservation 1: Nikolaj William Coster-Waldau (born 27 July 1970) is a Danish actor and producer. He graduated from the Danish National School of Performing Arts in Copenhagen in 1993,[1] and had his breakthrough role in Denmark with the film Nightwatch (1994). He played Jaime Lannister in the HBO fantasy drama series Game of Thrones, for which he received two Primetime Emmy Award nominations for Outstanding Supporting Actor in a Drama Series.. Coster-Waldau has appeared in numerous films in his native Denmark and Scandinavia, including Headhunters (2011) and A Thousand Times Good Night (2013). In the U.S, his debut film role was in the war film Black Hawk Down (2001), playing Medal of Honor recipient Gary Gordon.[2] He then played a detective in the short-lived Fox television series New Amsterdam (2008), and appeared in the 2009 Fox television film Virtuality, originally intended as a pilot.\nThought 2: Because he appeared in the 2009 Fox television film Virtuality, he should have worked with the Fox Broadcasting Company.\nAction 2: Finish[SUPPORTS]\n\nExample 2: Claim: Stranger Things is set in Bloomington, Indiana.\nThought 1: I should search for Stranger Things, and see if it is set in Bloomington, Indiana.\nAction 1: Search[Stranger Things]\nObservation 1: Stranger Things is an American science fiction horror drama television series created by the Duffer Brothers. Set in the 1980s, primarily in the fictional town of Hawkins, Indiana, the series centers on a number of mysteries and supernatural events occurring around the town and their impact on an ensemble of child and adult characters. \nThought 2: The observation says that it is set in a "fictional town of Hawkins, Indiana", so it is not set in Bloomington.\nAction 2: Finish[REFUTES]\n\nExample 3: Claim: Beautiful reached number two on the Billboard Hot 100 in 2003.?\nThought 1: I need to search the song Beautiful and find if it reached number two on the Billboard Hot 100 in 2003.\nAction 1: Search[Beautiful]\nObservation 1: Could not find [Beautiful]. Similar: [\'Beautiful\', \'Beautiful, Beautiful\', \'A Beautiful Mind (film)\', \'Beautiful (Christina Aguilera song)\', \'Life Is Beautiful\'].\nThought 2: From suggestions, I should search "Beautiful (Christina Aguilera song)" to find the song.\nAction 2: Search[Beautiful (Christina Aguilera song)]\nObservation 2: "Beautiful" is a song recorded by American singer Christina Aguilera for her fourth studio album, Stripped (2002).\nThought 3: It does not mention Billboard, so I need to look up "Billboard Hot 100" to find if it reached number two on it in 2003.\nAction 3: Lookup[Billboard Hot 100]\nObservation 3: (Result 1 / 3) The song peaked at number two on the Billboard Hot 100 in the United States, where it was certified Gold for 500,000 units shipped.\nThought 4: It only says the song peaked at number two on the Billboard Hot 100, but not if it was in 2003. I am not sure if this claim is true or not.\nAction 4: Finish[NOT ENOUGH INFO]\n\n'

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

# ReAct

In [ ]:
import json
import sys

def factCheck(claim, template, to_print=True):
    
    env.reset(claim)
    
    if to_print:
        print(claim)
    prompt = "{}Claim: {}".format(template, claim)
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        thought_action = llm(prompt + f"Thought {i}:")
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

In [ ]:
from server.models.factCheck.GPT_factCheck import get_completion

def is_drug(input):
  prompt = 'Is {} a name of any drug, chemical compound or vaccine? Answer only "Yes" or "No".'.format(input)
  answer = get_completion(prompt)
  if "Yes" in answer:
    return True
  return False
    
is_drug("Creatine")

In [ ]:
claim = "Creatine can cause abdominal cramp"
r, info = factCheck(claim, template)

In [ ]:
claim = "Pfizer vaccine can cause headache"
r, info = factCheck(claim, template)